# Chicago Analysis


In [ ]:
#| code-fold: true
#| code-summary: Imports
%matplotlib inline
import matplotlib_inline
import matplotlib.pyplot as plt
import geopandas as gpd
import warnings

import matplotlib.pyplot as plt

# from enrich.console import Console, get_theme
from ClimRR import get_logger, DATA_DIR, set_plot_style
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
set_plot_style()
log = get_logger('ClimRR')
from rich.console import Console as rConsole
from enrich.style import STYLES
from rich.theme import Theme

theme = Theme(STYLES)
log = get_logger('ClimRR')
console = rConsole(theme=theme, log_path=False, markup=True)

In [ ]:
from ClimRR import load_shapefile, load_csvs

shape = load_shapefile()
data = load_csvs(shape)

In [ ]:
square = shape[shape["Crossmodel"] == 'R146C497']
square.explore()

In [ ]:
import geodatasets
chipop = gpd.read_file(
    geodatasets.get_path('geoda.chicago_commpop')
).to_crs(square.crs)
chihealth = gpd.read_file(
    geodatasets.get_path('geoda.chicago_health')
).to_crs(square.crs)
chigroc = gpd.read_file(
    geodatasets.get_path('geoda.groceries')
).to_crs(square.crs)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
ax = chipop.to_crs(square.crs).plot(column="POP2010", legend=True, ax=ax)
ax.set_axis_off()
_ = ax.set_title(f"Chicago population by Neighborhood [2010]")

In [ ]:
chipop['boundary'] = chipop.boundary

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
ax = chipop.boundary.plot(linewidth=0.8, color='#838383', ax=ax)
ax.set_axis_off()
_ = ax.set_title('Chicago Neighborhoods')

In [ ]:
wtown = chipop[chipop["community"] == 'WEST TOWN']
humboldt = chipop[chipop["community"] == 'HUMBOLDT PARK']

In [ ]:
humboldt.explore()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
ax = humboldt.overlay(shape, how='intersection').plot(ax=ax, legend=True)
ax = (
    hp := chipop[chipop['community'] == 'HUMBOLDT PARK'].overlay(
        shape,
        how='intersection'
    )
).plot(ax=ax, legend=True)
ax = (
    lp := chipop[chipop['community'] == 'LINCOLN PARK'].overlay(
        shape,
        how='intersection'
    )
).plot(ax=ax, legend=True)
ax = chipop.boundary.plot(ax=ax, color='#666666', linewidth=0.8)
ax = lp.boundary.plot(color='red', ax=ax)
ax = hp.boundary.plot(color='red', ax=ax)
ax.set_axis_off()
_ = ax.set_title('Intersection of Humboldt Park & ClimRR data')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
chiwind = data['WindSpeed'].overlay(
    chipop,
    how='intersection'
).overlay(chipop, how='union')
ax = chiwind.boundary.plot(ax=ax, color='#666666', linewidth=0.8)
ax.set_axis_off()

In [ ]:
_, ax = plt.subplots(figsize=(10, 7))
ax = chipop.boundary.plot(color='#666666', linewidth=0.8, ax=ax, alpha=0.2)
ax = chiwind.plot(column='hist', ax=ax, legend=True)
ax.set_axis_off()
ax.set_title('Historical Wind Data across Chicago Neighborhoods')
plt.tight_layout()

In [ ]:
chiwind.explore(column='hist')

In [ ]:
_, ax = plt.subplots()
ax = chiwind.plot(column='hist', scheme='quantiles', k=8, ax=ax)
_ = ax.set_title('WindSpeed, historical')
ax.set_axis_off()
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots()
ax = chiwind.plot(column='rcp45_midc', scheme='quantiles', k=3, ax=ax)
ax.set_axis_off()
_ = ax.set_title('WindSpeed, Mid-Century [RCP45]')
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots()
ax = chiwind.plot(column='rcp45_endc', scheme='quantiles', k=3, ax=ax)
_ = ax.set_title('WindSpeed, End-Century [RCP45]')
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(16, 7))
ax0 = chiwind.plot('hist', ax=ax[0])
ax1 = chiwind.plot('rcp45_midc', ax=ax[1])
ax2 = chiwind.plot('rcp45_midc', ax=ax[2])
ax0.set_axis_off()
ax1.set_axis_off()
ax2.set_axis_off()

In [ ]:
data['WindSpeed'].shape

In [ ]:
selection = shape[0:5]

for index, row in selection.iterrows():
    # get the area of the polygon
    poly_area = row['geometry'].area
    console.print(f"Polygon area at {index} is {poly_area:.3f}")